<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [159]:
import pandas as pd
import jaydebeapi
import json

from datetime import datetime
import datetime as dt

In [110]:
path = "access to the database.txt"
with open(path) as f:
    access = json.load(f)
    
driver = 'ojdbc14.jar'
path_base = access['path_base']
password = access['password']
login = access['login']
port = access['port']
sid = access['sid']

conn = jaydebeapi.connect(
    'oracle.jdbc.driver.OracleDriver',
    f'jdbc:oracle:thin:{login}/{password}@{path_base}:{port}/{sid}',
    [login, password],
    driver)

curs = conn.cursor()

In [ ]:
def addTable():
    curs.execute(
        '''insert into uszn.temp$_vozv 
           (F, I, O, SUMA, PR)
            values(1, 'привет', '3', '4', '5')''')
    
def delTable():
    curs.execute('''delete from uszn.temp$_vozv''')    

In [ ]:
def ShowCountTable():
    curs.execute('''select count(*) from uszn.temp$_vozv''')
    print(curs.fetchall())
    
def ShowTable():
    curs.execute('''select * from uszn.tsrv_user_groups''')
    for row in curs.fetchall():
        print(row)
    


In [ ]:
ShowTable()

In [ ]:
delTable()

In [ ]:
curs.execute('''
    select *
        from uszn.temp$_vozv
    ''')
for row in curs.fetchall():
    print(row)

In [ ]:
curs.execute('''select count(*) from uszn.temp$_vozv''')
print(curs.fetchall())

In [44]:
def zags():
    curs.execute('''
    select 
       REPLACE(uszn.pkGen.BLOBToCLOB(msg.message_data, 'UTF8'),'<?xml version="1.0" encoding="UTF-8" ?>')
    from uszn.all_smev3_inc_messages msg
where
  message_kind_id=1 and
  data_kind_region_id=0 and
  data_kind_id in (
    /* старые форматы */ 16, 17, 18, 19, 20,
    /* новые форматы */ 28, 29, 30, 31, 32, 33) and
  Trunc(date_created, 'dd') between ADD_MONTHS(TRUNC(sysdate,'mm'),-1) and sysdate
order by date_created, id
    ''')
    with open('zagz.xml', 'w') as f:
        f.write('<?xml version="1.0" encoding="UTF-8" ?>')
        f.write('<SOAP-ENV:Envelope xmlns:SOAP-ENV="http://schemas.xmlsoap.org/soap/envelope/">')
        for row in curs.fetchall():
            f.write('\n')
#             f.write(row[0] )
            print(str(row[0][0]))
            n = input()
            if n == '1':
                break
        f.write('\n')
        f.write('</SOAP-ENV:Envelope>')

In [45]:
zags()

TypeError: 'oracle.sql.CLOB' object is not subscriptable

In [53]:
def alarm_ep():
    curs.execute(
    '''select count(*) from
     (select id
       from uszn.all_smev3_inc_messages
       where date_created>=To_Date('01.01.2021') and proc_status_id in (3))'''
    )
    cnt = int(curs.fetchall()[0][0])
    print(cnt)
    if cnt > 0:
        curs.execute(
        '''
        declare
          iiIDs        uszn.pkGen.TIntegers;
          i$ Pls_Integer;
        begin
        select id bulk collect into iiIDs
          from uszn.all_smev3_inc_messages
          where date_created>=To_Date('01.01.2021') and proc_status_id in (3);
          for i in 1..iiIDs.count loop
            begin
              i$ := uszn.pkSMEV3.ProcessInMessage(iiIDs(i), 1, 1,'Принудительная обработка входящего сообщения СМЭВ-3');
            commit;
            exception
              when Others then
                rollback;
                raise;
            end;
          end loop;
        end;
        '''
        )
        with open('alarm_ep.txt', 'w') as f:
            f.write('*************************************\n>')
            f,write(cnt)
            f.write('\n')
            f.write('*************************************\n>')

In [54]:
alarm_ep()

0


In [131]:
def GKH():
    curs.execute('''
select region_id,
       uszn.pkTSrv.GetRegionName(region_id),
       sum(fam_memb),
       sum(cnt),
       sum(area)
from
(select region_id,
       --n_row,
       Ceil(Sum(pc_count)) as fam_memb,
       Count(*) as cnt,
       Max(area) as area
   from (
     select data.region_id,
            data.coll_id,
            data.pc_count,
            case when levels.lvl=3 then 41 end as n_row,
            uszn.ToNumberDef(uszn.pkPerson.GetRawPCReqValueOnDate(data.region_id, data.coll_id, 3760, 3845, null, data.payout_date), 0) as area
        from (
          select id as lvl
            from uszn.u_dummy
            where id<=3) levels,
         (with mapped_pkafs as
             (select int_region_id as pkaf_region_id,
                     int_id as pkaf_id,
                     uszn.ToIntDef(ext_code) as cat_code
                 from uszn.dic_data_exchange_mappings
                 where kind_id=70 and
                       ext_region_id not in (71, 72) and
                       uszn.ToIntDef(ext_code) not in (40, 50) and
                       ((uszn.ToIntDef(ext_code) between 10 and 150) or (uszn.ToIntDef(ext_code) between 340 and 351) ))
select a.region_id,
    a.pka_people_coll_id as people_id,
    a.poi_payout_date as payout_date,
    a.pc_count_applied_to as pc_count,
    uszn.pkOutDocCol.GetPFRF_Cat( a.region_id, a.pka_people_coll_id, a.pkaf_region_id, a.pkaf_id, Trunc(a.poi_payout_date, 'mm'),
    Last_Day(a.poi_payout_date)) as n_cat, uszn.pkPic.GetCollByRole(a.region_id, a.pka_people_coll_id, 46, a.poi_payout_date, 0, 0) as coll_id
 from uszn.all_po_amounts a
 where (a.region_id, a.id) in (
    select a.region_id,
           First_Value(a.id) over (partition by a.region_id, a.poi_assigned_id order by a.poi_payout_date desc, a.pc_count_applied_to desc, a.income_date desc, a.id) as amount_id
        from uszn.all_po_amounts a
        where a.region_id not in (71, 72) and
              (a.pka_kind_region_id, a.pka_kind_id) in ((104,29)) and
              a.finsrc_region_id=0 and a.finsrc_id=1 and a.status_kind_id=2 and
              a.poi_payout_date between TRUNC(ADD_MONTHS(SYSDATE,-1),'MM') and TRUNC(SYSDATE,'MM')-1 and
              (((a.pkaf_region_id, a.pkaf_id) in ((104, 339)) and
              Exists( select 1
                        from uszn.r_personal_doc_instances d,
                             mapped_pkafs m
                           where d.region_id=a.region_id and d.people_coll_id=a.pka_people_coll_id and
                                 d.class_id=7116 and m.pkaf_region_id*1000000+m.pkaf_id=uszn.ToIntDef(d.value) ))
                                 or
                                 (a.pkaf_region_id, a.pkaf_id) in (select pkaf_region_id, pkaf_id from mapped_pkafs) )))data)
                                 where n_row is not null
                                 group by n_row,
                                       region_id, coll_id)
group by region_id''')
#     data = pd.DataFrame()
    data = {
         'id МО' : [] ,
         'Название МО' : [] ,
         'Всего' : [] ,
         'в том числе льготники' : [] ,
         'площадь' : []             
        }
    for row in curs.fetchall():
#         print(row[0], row[1], row[2], row[3], row[4])
        data['id МО'].append(row[0])
        data['Название МО'].append(row[1])
        data['Всего'].append(row[2])
        data['в том числе льготники'].append(row[3])
        data['площадь'].append(row[4])
        
    df = pd.DataFrame(data)
    
    today = dt.date.today()
    first = today.replace(day=1)
    lastMonth = first - dt.timedelta(days=1)
    date_report = lastMonth.strftime('%m.%Y')
    
    file_name = f'отчет ЖКВ за {date_report} в разрезе МО'+ '.xlsx'
    df.to_excel(file_name, index=False)
        
                 

In [132]:
GKH()

In [122]:
data

{'id МО': ['58', 'row[0]'],
 'Название МО': ['row[1]'],
 'Всего': [],
 'в том числе льготники': [],
 'площадь': []}

In [160]:
d = datetime.now().strftime('%d.%m.%Y')
year = datetime.now().strftime('%Y')
month = datetime.now().strftime('%m')

today = dt.date.today()
first = today.replace(day=1)
lastMonth = first - dt.timedelta(days=1)
date_report = lastMonth.strftime('%m.%Y')

In [162]:
file_name = f'отчет ЖКВ за {date_report} в разрезе МО'
print(file_name)

отчет ЖКВ за 03.2022 в разрезе МО


In [164]:
today = dt.date.today()
first = today.replace(day=1)
lastMonth = first - dt.timedelta(days=1)
print(lastMonth.strftime('%m.%Y'))

03.2022


In [152]:
print(today)

2022-04-16


In [153]:
print(first)

2022-04-01


In [154]:
print(datetime.timedelta(days=1))

1 day, 0:00:00


In [171]:
print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

2022-04-16 16:40:20


In [174]:
for region_id in range(58,71):
    print(region_id)
    for row in range(1,4):
        name_collection = f"'ЖКВ-{row}'"
        print(name_collection)

58
'ЖКВ-1'
'ЖКВ-2'
'ЖКВ-3'
59
'ЖКВ-1'
'ЖКВ-2'
'ЖКВ-3'
60
'ЖКВ-1'
'ЖКВ-2'
'ЖКВ-3'
61
'ЖКВ-1'
'ЖКВ-2'
'ЖКВ-3'
62
'ЖКВ-1'
'ЖКВ-2'
'ЖКВ-3'
63
'ЖКВ-1'
'ЖКВ-2'
'ЖКВ-3'
64
'ЖКВ-1'
'ЖКВ-2'
'ЖКВ-3'
65
'ЖКВ-1'
'ЖКВ-2'
'ЖКВ-3'
66
'ЖКВ-1'
'ЖКВ-2'
'ЖКВ-3'
67
'ЖКВ-1'
'ЖКВ-2'
'ЖКВ-3'
68
'ЖКВ-1'
'ЖКВ-2'
'ЖКВ-3'
69
'ЖКВ-1'
'ЖКВ-2'
'ЖКВ-3'
70
'ЖКВ-1'
'ЖКВ-2'
'ЖКВ-3'
